In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('resources/filtered_train.csv')

In [3]:
df.head()

,restaurant_id,user_id,score
0,2,14,1
1,2,2087,1
2,2,4625,1
3,2,4626,1
4,2,4826,1


In [4]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446845 entries, 0 to 446844
Data columns (total 3 columns):
restaurant_id    446845 non-null int64
user_id          446845 non-null int64
score            446845 non-null int64
dtypes: int64(3)
memory usage: 10.2 MB


In [5]:
df = df.apply(pd.to_numeric,downcast='unsigned')

In [6]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446845 entries, 0 to 446844
Data columns (total 3 columns):
restaurant_id    446845 non-null uint32
user_id          446845 non-null uint32
score            446845 non-null uint8
dtypes: uint32(2), uint8(1)
memory usage: 3.8 MB


In [7]:
n_users = df['user_id'].nunique()
n_restaurants = df['restaurant_id'].nunique()
print(n_users)
print(n_restaurants)

90340
3368


In [8]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

In [9]:
user_id_c = CategoricalDtype(sorted(df.user_id.unique()), ordered=True)
restaurant_id_c = CategoricalDtype(sorted(df.restaurant_id.unique()), ordered=True)
value = df['score']
row = df.user_id.astype(user_id_c).cat.codes
col = df.restaurant_id.astype(restaurant_id_c).cat.codes

In [10]:
sparse_matrix = csr_matrix((value, (row, col)), shape=(user_id_c.categories.size, restaurant_id_c.categories.size))

In [14]:
sparse_matrix = sparse_matrix.astype('f')

In [15]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(sparse_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)